# Data

In [ ]:
import numpy as np 
import pandas as pd
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv ("../input/cardiovascular-disease-dataset/cardio_train.csv",delimiter =';')
df.drop('id', inplace=True, axis=1)
df.head(1)

# label encoder

In [ ]:
label_encoder = LabelEncoder()

def encode(col,n_bins):
    x = label_encoder.fit_transform(pd.cut(col, n_bins, retbins=True)[0])
    return x
    
for i in df.columns:
    if(len(df[i].unique()) > 3):
        n_bins = 2
        df[i]= encode(df[i],n_bins)
    else:
        n_bins = len(df[i].unique())
        df[i]= encode(df[i],n_bins)

for i in df.columns:
    print(i,len(df[i].unique()))

In [ ]:
split= int(df.shape[0]*90/100)
df_test = df.iloc[split:,:]
df = df.iloc[:split,:]
print(df.shape)
print(df_test.shape)

# Gini_active

In [ ]:
def GINI(DF,column):
    n_bins = len(DF[column].unique())
    DF[column] = encode(DF[column],n_bins)
    values = np.sort(DF[column].unique())
    Ginis_matrix  = np.zeros( (2 , len(values)))
    sums = np.zeros(len(values))
    Ginis= np.zeros(len(values))
     
    for j in values:
        for i in [0,1]:
            Ginis_matrix[i,j] = (DF.loc[((DF.cardio==i) & (DF[column]==j)) ]).shape[0]
         
    sums = sum(Ginis_matrix)
    
    for i in values:
        Ginis[i] = 1-(Ginis_matrix[1][i]/sums[i])**2- (Ginis_matrix[0][i]/sums[i])**2
    
    Gini_out = np.sum(Ginis*sums/DF.shape[0])
    return Gini_out,Ginis_matrix


In [ ]:
def node_gini(df,l):
    if(len(l) == 0 or df.shape[0] == 0):
        return 'NONE'
    g={}
    for i in l:
        g[i],GINIS = GINI(df,i)
    nodee = min(g, key=g.get)
    gini_value= g[min(g, key=g.get)]
    return nodee,gini_value

In [ ]:
def newListRemove(element, l): 
    new_l=[]
    for i in l:
        if(i!=element):
            new_l.append(i)
    return new_l

In [ ]:
def Count (frame):
    yes=no=0
    counts=[]
    frame=frame.reset_index()
    for row in range(frame.shape[0]):
        if(frame["cardio"][row]==1):
            yes+=1
        else:
            no+=1
    counts.append(no)
    counts.append(yes)
    return counts , counts.index(max(counts))

# CART

In [ ]:
def update(f,l,n):
    if(len(l) == 0 or f.shape[0] == 0 or n =='NONE'):
        return pd.DataFrame(),[]
    frames=[]
    for i in f[n].unique():
        frames.append( f[f[n]== i] )
    l = newListRemove(n,l)
    return frames,l        

In [ ]:
class desicion_Node:
    def __init__(self ,  nodes_list , gini_attr,gini_value):
        self.nodes = nodes_list
        self.attr = gini_attr
        self.gini_split=gini_value
class Leaf:
    def __init__(self, rows):
        self.attr,self.dec= Count(rows)   

In [ ]:
def insert(data,l, gini):
    if(len(l) == 0 or data.shape[0] == 0 or gini < 0.4 ):
        return Leaf(data)

    gini_split, gini_value = node_gini(data,l)
    data_list,l = update(data,l, gini_split )
    nodes_list=[]
    
    for i in range(len(data_list)):
        nodes_list.append( insert( data_list[i] , l ,gini_value ))
    
    return desicion_Node( nodes_list ,gini_split,gini_value)

In [ ]:
l=list(df.columns)
l.remove("cardio")
root = insert(df,l ,100)

# Print & decision

In [ ]:
def print_marina(node,space):
    if ( (isinstance(node.attr, str))  == False):
        print(space,"Leaf:", node.attr ,",Decision: " ,node.dec)
        return 
    print(space,node.attr , node.gini_split)
    for i in range(len(node.nodes)):
        print_marina(node.nodes[i],space+"   ")

In [ ]:
print_marina(root,"")

# Test

In [ ]:
def decision_fn(row,node):
    if ( (isinstance(node.attr, str))  == False):
        return node.dec
    try:
        return decision_fn(row,node.nodes[int(row[node.attr])])
    except:
        return decision_fn(row,node.nodes[0])

In [ ]:
acc=0
for i in range(df_test.shape[0]):
    row = df_test.iloc[i,:]
    des = decision_fn( row ,root)
    if(int(row['cardio'])==des):
        acc+=1
        
print(acc/df_test.shape[0]*100)